In [1]:
# part1
# mathematical expression
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
    return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

# Example usage
print(f(2, 3, 4))  

6.336362190988558


In [2]:
# Part 2
# Analytical Gradient
def gradf(a, b, c):
    dfd_a = -3 * a**2 - 0.5 * a**(-0.5)
    dfd_b = 3 * cos(3 * b) + 2.5 * b**1.5
    dfd_c = 1.0 / (c**2)
    return [dfd_a, dfd_b, dfd_c]

# Expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
    ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [3]:
# part 3
# according to video
import math
def f(a, b, c):
    return -a**3 + math.sin(3*b) - 1.0/c + b**2.5 - a**0.5

a, b, c = 2.0, 3.0, 4.0
h = 1e-5
numerical_grad = [0, 0, 0]

# Compute numerical gradients
for dim, (val, grad) in enumerate(zip([a, b, c], numerical_grad)):
    if dim == 0:
        fxh = f(a + h, b, c)
        f_original = f(a, b, c)
    elif dim == 1:
        fxh = f(a, b + h, c)
        f_original = f(a, b, c)
    elif dim == 2:
        fxh = f(a, b, c + h)
        f_original = f(a, b, c)
    
    numerical_grad[dim] = (fxh - f_original) / h

# Expected gradient for verification
ans = [-3*a**2 - 0.5*a**(-0.5), 3*math.cos(3*b) + 2.5*b**1.5, 1.0/(c**2)] 

for dim in range(3):
    ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

WRONG! for dim 0: expected -12.353553390593273, yours returns -12.353612948956536
WRONG! for dim 1: expected 10.25699027111255, yours returns 10.257004202163245
OK for dim 2: expected 0.0625, yours returns 0.06249984378925432


In [4]:
# Part 3
# Numerical Gradient
h=1e-5
def numerical_gradient(a, b, c):
    dfd_a = (f(a + h, b, c) - f(a - h, b, c)) / (2 * h)
    dfd_b = (f(a, b + h, c) - f(a, b - h, c)) / (2 * h)
    dfd_c = (f(a, b, c + h) - f(a, b, c - h)) / (2 * h)
    return [dfd_a, dfd_b, dfd_c]

numerical_grad = numerical_gradient(2, 3, 4)

for dim in range(3):
    ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390820336
OK for dim 1: expected 10.25699027111255, yours returns 10.256990271617639
OK for dim 2: expected 0.0625, yours returns 0.06250000001983835


In [5]:
# Part 3
# Symmetric Numerical Gradient
hh = 1e-5
a=2
b=3
c=4
# Compute the second symmetric derivative for each dimension
numerical_grad2 = [
    (f(a + h, b, c) - 2 * f(a, b, c) + f(a - h, b, c)) / h**2,
    (f(a, b + h, c) - 2 * f(a, b, c) + f(a, b - h, c)) / h**2,
    (f(a, b, c + h) - 2 * f(a, b, c) + f(a, b, c - h)) / h**2
]

#numerical_grad2 = [0, 0, 0]

# Compare with expected answer
for dim in range(3):
    ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

WRONG! for dim 0: expected -12.353553390593273, yours returns -11.911627240124288
WRONG! for dim 1: expected 10.25699027111255, yours returns 2.7861091211889284
WRONG! for dim 2: expected 0.0625, yours returns -0.031246116805050402


In [6]:
# section 2
# sopport for softmax

from math import exp, log

class Value:
  
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def __sub__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data - other.data, (self, other), '-')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += -1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __truediv__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data / other.data, (self, other), '/')
        
        def _backward():
            self.grad += (1 / other.data) * out.grad
            other.grad += (-self.data / (other.data ** 2)) * out.grad
        out._backward = _backward
        
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data ** other, (self,), f'**{other}')
        
        def _backward():
            self.grad += (other * (self.data ** (other - 1))) * out.grad
        out._backward = _backward
        
        return out
    
    def exp(self):
        out = Value(exp(self.data), (self,), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
    
    def log(self):
        out = Value(log(self.data), (self,), 'log')
        
        def _backward():
            self.grad += (1 / self.data) * out.grad
        out._backward = _backward
        
        return out
    
    def backward(self):
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


In [7]:
from math import exp, log, sin, cos

class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other): 
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __radd__(self, other):
    return self + other

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    
    return out

  def __rmul__(self, other):
    return self * other

  def __sub__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data - other.data, (self, other), '-')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += -1.0 * out.grad
    out._backward = _backward
    
    return out

  def __rsub__(self, other):
    return Value(other) - self

  def __truediv__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data / other.data, (self, other), '/')
    
    def _backward():
      self.grad += (1 / other.data) * out.grad
      other.grad += (-self.data / (other.data ** 2)) * out.grad
    out._backward = _backward
    
    return out

  def __rtruediv__(self, other):
    return Value(other) / self

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data ** other, (self,), f'**{other}')
    
    def _backward():
      self.grad += (other * self.data ** (other - 1)) * out.grad
    out._backward = _backward
    
    return out

  def __neg__(self):
    return self * -1

  def exp(self):
    out = Value(exp(self.data), (self,), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward
    
    return out

  def log(self):
    out = Value(log(self.data), (self,), 'log')
    
    def _backward():
      self.grad += (1 / self.data) * out.grad
    out._backward = _backward
    
    return out

  def sin(self):
    out = Value(sin(self.data), (self,), 'sin')
    
    def _backward():
      self.grad += cos(self.data) * out.grad
    out._backward = _backward
    
    return out

  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

# Softmax function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# Example usage
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print("loss: ",loss.data)

function_answer=[0,0,0,0]

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  function_answer[dim]=logits[dim].grad
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

loss:  2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.886450380640099


In [10]:
# comparing
from nn_torch import torch_answer
d=1e-7
dif=0
print("comparing answers of my function and torch answers:")
for dim in range(4):
  diffrence= abs(function_answer[dim]-torch_answer(dim))
  if diffrence< d:
    dif="less"
  else:
    dif="more"

  print(f"my function[{dim}]: {function_answer[dim]}, and torch_answer[{dim}]: {torch_answer(dim)}, diffrence is {dif} than {d}" )


comparing answers of my function and torch answers:
my function[0]: 0.041772570515350445, and torch_answer[0]: 0.041772566735744476, diffrence is less than 1e-07
my function[1]: 0.8390245074625319, and torch_answer[1]: 0.8390244245529175, diffrence is less than 1e-07
my function[2]: 0.005653302662216329, and torch_answer[2]: 0.005653302185237408, diffrence is less than 1e-07
my function[3]: -0.886450380640099, and torch_answer[3]: -0.8864503502845764, diffrence is less than 1e-07
